In [1]:
import time
import requests
import io
import pdfplumber
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import logging

# Configurar o logging
logging.basicConfig(filename="extracao_notas_teste.log", level=logging.INFO, 
                    format="%(asctime)s - %(message)s")

# Definir o caminho fixo para salvar o CSV
output_csv = r"C:\Users\Sóstenes\OneDrive - Insper - Instituto de Ensino e Pesquisa\Documentos\Projeto diss\e-natjus-scraper\notas_enteral_5.csv"

# Função para limpar valores problemáticos
def limpar_valor(valor):
    valor = valor.replace("$1^{\text {a }}$", "1ª").replace("$8^{\text {a }}$", "8ª")
    if valor.startswith("1") and len(valor) <= 4 and valor.isalnum():
        valor = "I" + valor[1:]
    if valor.lower() in [".", "-", "$x$", "x", ""]:
        return "Não informado"
    return valor

# Função para extrair dados do HTML
def extrair_dados_html(driver):
    dados = {}
    wait = WebDriverWait(driver, 10)
    try:
        elementos = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'row')]/div")))
        for elem in elementos:
            texto = elem.text.strip()
            if ":" in texto:
                chave, valor = texto.split(":", 1)
                dados[chave.strip()] = limpar_valor(valor.strip())
        return dados
    except Exception as e:
        logging.warning(f"Falha ao extrair dados do HTML: {e}")
        return {}

# Função para extrair dados do PDF
def extrair_dados_pdf(pdf):
    texto_completo = ""
    for page in pdf.pages:
        texto_completo += (page.extract_text() or "") + "\n"
    
    nota_dados = {"URL": "", "Link PDF": ""}
    linhas = texto_completo.split("\n")
    ultima_chave = None
    contador_tecnologia = 0
    
    for linha in linhas:
        linha = linha.strip()
        if not linha:
            continue
        if ":" in linha:
            chave, valor = linha.split(":", 1)
            chave = chave.strip()
            valor = limpar_valor(valor.strip())
            if valor == "Não informado":
                logging.info(f"Nota {nota_dados.get('URL', 'desconhecida')} - Campo '{chave}' ausente ou inconsistente")
            if chave == "Tecnologia":
                contador_tecnologia += 1
                chave = f"Tecnologia_{contador_tecnologia}"
            if chave in nota_dados:
                if isinstance(nota_dados[chave], list):
                    nota_dados[chave].append(valor)
                else:
                    nota_dados[chave] = [nota_dados[chave], valor]
            else:
                nota_dados[chave] = valor
            ultima_chave = chave
        elif ultima_chave and linha:
            if isinstance(nota_dados[ultima_chave], list):
                nota_dados[ultima_chave][-1] += " " + linha
            else:
                nota_dados[ultima_chave] += " " + linha
    
    # Campos desejados
    campos_desejados = {
        "URL": "URL",
        "Link PDF": "Link PDF",
        "Data de conclusão": "Data de conclusão",
        "Idade": "Idade",
        "Sexo": "Sexo",
        "Cidade": "Cidade",
        "Nome do Advogado": "Nome do Advogado",
        "Número OAB": "Número OAB",
        "Autor está representado por": "Autor está representado por",
        "Esfera/Órgão": "Esfera/Órgão",
        "Vara/Serventia": "Vara/Serventia",
        "CID": "CID",
        "Diagnóstico": "Diagnóstico",
        "Meio(s) confirmatório(s) do diagnóstico já realizado(s)": "Meio(s) confirmatório(s)",
        "Tipo da Tecnologia": "Tipo da Tecnologia",
        "Situação do registro": "Situação do registro",
        "Descrição": "Descrição",
        "Tecnologia_1": "Tecnologia_1",
        "Descrever as opções disponíveis no SUS e/ou Saúde Suplementar": "Opções SUS/Saúde Suplementar",
        "Tecnologia_2": "Tecnologia_2",
        "Custo da tecnologia": "Custo da tecnologia",
        "Fonte do custo da tecnologia": "Fonte do custo",
        "Tecnologia_3": "Tecnologia_3",
        "Evidências sobre a eficácia e segurança da tecnologia": "Evidências eficácia/segurança",
        "Benefício/efeito/resultado esperado da tecnologia": "Benefício esperado",
        "Recomendações da CONITEC para a situação clínica do demandante": "Recomendações CONITEC",
        "Tecnologia_4": "Tecnologia_4",
        "Conclusão Justificada": "Conclusão Justificada",
        "Conclusão": "Conclusão",
        "Referências bibliográficas": "Referências",
        "NatJus Responsável": "NatJus Responsável",
        "Instituição Responsável": "Instituição Responsável",
        "Outras Informações": "Outras Informações",
        "Registro na ANVISA?": "Registro na ANVISA?",
        "Há evidências científicas?": "Há evidências científicas?",
        "Justifica-se a alegação de urgência, conforme definição de Urgência e Emergência do CFM?": "Urgência CFM"
    }
    
    dados_filtrados = {}
    for chave_orig, chave_nova in campos_desejados.items():
        dados_filtrados[chave_nova] = nota_dados.get(chave_orig, "Não informado")
    
    # Corrigir a separação da Conclusão
    conclusao = dados_filtrados["Conclusão"]
    if "Há evidências científicas?" in conclusao:
        inicio = conclusao.index("Há evidências científicas?")
        evidencias_part = conclusao[inicio:].split(" ", 4)
        evidencias = " ".join(evidencias_part[3:]) if len(evidencias_part) > 3 else "Não informado"
        if "Não se aplica" in evidencias:
            evidencias = "Não se aplica"
        elif "Sim" in evidencias:
            evidencias = "Sim"
        elif "Não" in evidencias:
            evidencias = "Não"
        dados_filtrados["Há evidências científicas?"] = evidencias
        dados_filtrados["Conclusão"] = conclusao[:inicio].strip()
    if "Justifica-se a alegação de urgência" in conclusao:
        inicio = conclusao.index("Justifica-se a alegação de urgência")
        urgencia = conclusao[inicio:].split(" ")[-1]
        dados_filtrados["Urgência CFM"] = urgencia
        dados_filtrados["Conclusão"] = conclusao[:inicio].strip()
    
    return dados_filtrados

# Função para inicializar o driver
def iniciar_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-download-notifications")
    prefs = {
        "download.prompt_for_download": False,
        "download.default_directory": r"C:\Temp",
        "plugins.always_open_pdf_externally": False
    }
    options.add_experimental_option("prefs", prefs)
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

# Função para fazer a consulta inicial
def fazer_consulta(driver):
    logging.info("Acessando a página inicial...")
    print("Acessando a página inicial...")
    driver.get("https://www.pje.jus.br/e-natjus/pesquisaPublica.php")
    wait = WebDriverWait(driver, 30)
    
    logging.info("Preenchendo o formulário...")
    print("Preenchendo o formulário...")
    select_element = wait.until(EC.presence_of_element_located((By.NAME, "selTipoTecnologia")))
    select = Select(select_element)
    select.select_by_value("3")  # Selecionar "Produto" (valor 3)
    
    descricao_element = wait.until(EC.visibility_of_element_located((By.NAME, "txtProduto")))
    descricao_element.send_keys("dieta enteral")
    
    botao_pesquisar = wait.until(EC.element_to_be_clickable((By.ID, "btnPesquisar")))
    botao_pesquisar.click()
    
    logging.info("Esperando a lista de resultados...")
    print("Esperando a lista de resultados...")
    wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@class, 'btn-success') and @title='Nota Técnica']")))
    logging.info("Lista de resultados carregada.")
    print("Lista de resultados carregada.")
    return wait

# Função para processar uma nota em nova guia
def processar_nota(driver, wait, link_nota, nota_idx):
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[1])
    driver.get(link_nota)
    
    logging.info(f"Extraindo nota {nota_idx} - Link: {link_nota}")
    print(f"Extraindo nota {nota_idx} - Link: {link_nota}")
    
    dados_html = extrair_dados_html(driver)
    dados_completos = dados_html.copy()
    dados_completos["URL"] = link_nota
    
    campos_obrigatorios = ["Data de conclusão", "Idade", "Sexo", "Cidade", "CID", "Diagnóstico", "Tecnologia_1", "Conclusão"]
    faltam_dados = any(dados_html.get(campo, "Não informado") == "Não informado" for campo in campos_obrigatorios)
    
    if faltam_dados:
        try:
            botao_pdf = wait.until(EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'PDF Nota Técnica')]")))
            link_pdf = botao_pdf.get_attribute("href")
            logging.info(f"Link do PDF encontrado: {link_pdf}")
            print(f"Link do PDF encontrado: {link_pdf}")
            
            cookies = driver.get_cookies()
            session = requests.Session()
            for cookie in cookies:
                session.cookies.set(cookie['name'], cookie['value'])
            
            response = session.get(link_pdf, stream=True)
            response.raise_for_status()
            
            with pdfplumber.open(io.BytesIO(response.content)) as pdf:
                dados_pdf = extrair_dados_pdf(pdf)
                dados_completos.update(dados_pdf)
                dados_completos["Link PDF"] = link_pdf
            logging.info("Dados extraídos do PDF com sucesso.")
            print("Dados extraídos do PDF com sucesso.")
        except Exception as e:
            logging.error(f"Erro ao processar PDF: {e}")
            print(f"Erro ao processar PDF: {e}")
            dados_completos["Erro"] = str(e)
    else:
        logging.info("Dados extraídos do HTML com sucesso.")
        print("Dados extraídos do HTML com sucesso.")
    
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    return dados_completos

# Inicializar o driver
driver = iniciar_driver()

# Lista para armazenar os dados extraídos
dados_notas = []
notas_processadas = 0
LIMITE_NOTAS = 5  # Limitar a 5 notas

# Fazer a consulta inicial
wait = fazer_consulta(driver)

# Processar apenas a primeira página até o limite de 5 notas
logging.info("Processando os primeiros 5 resultados da página 1...")
print("\nProcessando os primeiros 5 resultados da página 1...")

botoes_nota = driver.find_elements(By.XPATH, "//a[contains(@class, 'btn-success') and @title='Nota Técnica']")
total_notas_pagina = len(botoes_nota)
logging.info(f"Total de notas na página 1: {total_notas_pagina}")
print(f"Total de notas na página 1: {total_notas_pagina}")

if total_notas_pagina == 0:
    logging.warning("Nenhuma nota encontrada na página 1.")
    print("Nenhuma nota encontrada na página 1.")
else:
    for i, botao in enumerate(botoes_nota[:LIMITE_NOTAS]):  # Limitar aos 5 primeiros
        if notas_processadas >= LIMITE_NOTAS:
            break
        link_nota = botao.get_attribute("href")
        dados_nota = processar_nota(driver, wait, link_nota, notas_processadas + 1)
        dados_notas.append(dados_nota)
        notas_processadas += 1
        
        wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@class, 'btn-success') and @title='Nota Técnica']")))

logging.info(f"Processamento concluído. Total de notas processadas: {notas_processadas}")
print(f"Processamento concluído. Total de notas processadas: {notas_processadas}")

# Criar DataFrame e salvar
df = pd.DataFrame(dados_notas)
df.to_csv(output_csv, index=False)

# Fechar o navegador
driver.quit()

print(f"\nDados salvos em: {output_csv}")
print("\nTotal de notas processadas:", len(df))
print("\nAmostra dos dados:")
print(df.head())

Acessando a página inicial...
Preenchendo o formulário...
Esperando a lista de resultados...
Lista de resultados carregada.

Processando os primeiros 5 resultados da página 1...
Total de notas na página 1: 34
Extraindo nota 1 - Link: https://www.pje.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=269061
Link do PDF encontrado: https://www.pje.jus.br/e-natjus/notaTecnica-dados.php?output=pdf&token=nt:269061:1743081662:64876d006e79dc359846a6963efd7301d40ba02eecb01cdb407c3f62429c23fe
Dados extraídos do PDF com sucesso.
Extraindo nota 2 - Link: https://www.pje.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=311661
Link do PDF encontrado: https://www.pje.jus.br/e-natjus/notaTecnica-dados.php?output=pdf&token=nt:311661:1743081664:cefc32a24c405e0763ee803151f7bf6c522383e378dd040e93e3843152b07bae
Dados extraídos do PDF com sucesso.
Extraindo nota 3 - Link: https://www.pje.jus.br/e-natjus/notaTecnica-dados.php?idNotaTecnica=282103
Link do PDF encontrado: https://www.pje.jus.br/e-natjus/n